In [1]:
import networkx as nx
import numpy as np
from scipy.special import binom
import copy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing

np.random.seed(1234)

## Functions

### About Networks and Adjacency matrices

In [2]:
def corruptAdjacencyMatrix(A, p, mode):
    """
    Corrupts the adjacency matrix A by adding missing and/or spurious edges
    """
    assert mode in ['missing', 'spurious', 'both'], "Invalid mode"

    if mode == 'missing':
        mask = np.random.choice([0, 1], size=A.shape, p=[p, 1-p])
        A_corrupted = np.multiply(A, mask)
    elif mode == 'spurious':
        mask = np.random.choice([0, 1], size=A.shape, p=[1-p, p])
        A_corrupted = np.maximum(A, mask)
    elif mode == 'both':
        mask_missing = np.random.choice([0, 1], size=A.shape, p=[p, 1-p])
        mask_spurious = np.random.choice([0, 1], size=A.shape, p=[1-p, p])
        A_corrupted = np.multiply(A, mask_missing) + mask_spurious
        A_corrupted = (A_corrupted > 0).astype(int)

    # Ensure A = A.T
    A_corrupted = np.maximum(A_corrupted, A_corrupted.T)
    
    return A_corrupted


def linksBetween(A, groups, g1, g2):
    """
    Compute the number of links between two groups
    """
    l = 0
    for i in groups[g1]:
        for j in groups[g2]:
            l += A[i, j]
    return l


def maxBetween(groups, g1, g2):
    """
    Compute the maximum number of links between two groups
    """
    return len(groups[g1]) * len(groups[g2])


def findGroup(groups, i):
    """
    Find the group of a given node
    """
    for g, group in enumerate(groups):
        if i in group:
            return g
    
    return None


def swap(groups, i, g_new):
    """
    Swap a node i from group to group g
    """
    # Copy the groups data structure
    new_groups = copy.deepcopy(groups)
    
    # Swap the node
    g_old = findGroup(groups, i)
    new_groups[g_old].remove(i)
    new_groups[g_new].append(i)

    return new_groups


def getLinkLists(A):
    """
    Get the list of links and not links from the adjacency matrix
    """
    links = []
    not_links = []

    for i in range(N):
        for j in range(i, N):
            if A[i, j] == 1:
                links.append((i, j))
            else:
                not_links.append((i, j))

    return links, not_links

### About sampling

In [3]:
def hamiltonian(A, groups):
    """
    Compute the Hamiltonian of the system
    """
    def h(A, groups, g1, g2):
        r = maxBetween(groups, g1, g2)
        l = linksBetween(A, groups, g1, g2)
        return np.log(r+1) + np.log(binom(r, l)+1)
            
    H = 0
    for g2 in range(N):
        for g1 in range(g2+1):
            H += h(A, groups, g1, g2)
    
    return H


def singleStep(groups, H, A):
    """
    Perform a single step of the Metropolis-Hastings algorithm
    """
    # Randomly select a node and a group
    i = np.random.randint(0, N)
    g_prop = np.random.randint(0, N)

    # Move the node to another group
    groups_prop = swap(groups, i, g_prop)
    
    # Compute the Hamiltonian of the new configuration
    H_prop = hamiltonian(A, groups_prop)

    # Acceptance probability
    if H_prop <= H:
        groups = groups_prop
        H = H_prop
    else:
        r = np.random.rand()
        if r < np.exp(H - H_prop):
            groups = groups_prop
            H = H_prop

    return groups, H


def samplingBranch(A, groups, n_samples, delay, seed, return_H=False):
    """
    Sampling algorithm (at equilibrium) to parallelize the computation
    """
    np.random.seed(seed)
    

    partitions_set = []
    hamiltonians_list = []

    H = hamiltonian(A, groups)

    for k in range(n_samples):
        for _ in range(delay):
            groups, H = singleStep(groups, H, A)
        partitions_set.append(groups)
        hamiltonians_list.append(H)

    if return_H:
        return partitions_set, hamiltonians_list
    else:
        return partitions_set
    

def parallelPartitionsSet(A, groups_init, n_samples, delay, n_cores=-1, return_H=True):
    """
    Get the set of partitions after the transient (at equilibrium)
    """
    if n_cores == -1:
       n_cores = multiprocessing.cpu_count()

    n_samples = int(n_samples) # Cast n_samples
    n_samples_per_core = n_samples//n_cores
    # if (n_samples_per_core*n_cores < n_samples):
    #     print("Warning: n_samples is not a multiple of n_cores, computing ", n_samples_per_core*n_cores, " samples instead.")

    seeds = np.random.randint(0, 2**32-1, n_cores) # Random seeds to avoid the same partitions
    input = [(A, groups_init, n_samples_per_core, delay, seeds[i], return_H) for i in range(n_cores)]

    with multiprocessing.Pool(processes=n_cores) as pool:
        results = pool.starmap(samplingBranch, input)

    if return_H:
        partitions_set = [item for sublist in [results[i][0] for i in range(n_cores)] for item in sublist]
        hamiltonians_list = [item for sublist in [results[i][1] for i in range(n_cores)] for item in sublist]
        return partitions_set, hamiltonians_list
    else:
        partitions_set = [item for sublist in results for item in sublist]
        return partitions_set
    

def generatePartitionsSet(A, n_samples, delay, transient, n_cores=-1, return_H=True):
    """
    Generate the set of partitions from the adjacency matrix A
    """
    # Groups data structure: list of lists
    N = A.shape[0]
    groups = [[] for _ in range(N)]

    # Uniformly random initialization
    for i in range(N): 
        g = np.random.randint(0, N)
        groups[g].append(i)

    # Transient
    H = hamiltonian(A, groups)
    for _ in range(transient):
        groups, H = singleStep(groups, H, A)
    
    return parallelPartitionsSet(A, groups, n_samples, delay, n_cores, return_H)

### About reliability

In [4]:
def singleLinkReliability(A, groups, i, j, H=None, return_H=False):
    """
    Reliability of a given link for a particular configuration
    """
    g_i = findGroup(groups, i)
    g_j = findGroup(groups, j)

    r = maxBetween(groups, g_i, g_j)
    l = linksBetween(A, groups, g_i, g_j)

    if H is None: H = hamiltonian(A, groups) # Possibly to save computation

    if return_H:
        return (l + 1)*np.exp(-H)/(r + 2), H
    else:
        return (l + 1)*np.exp(-H)/(r + 2)
    

def computeLinkReliabilityMatrix(A, partitions_set, hamiltionian_list=None):
    """
    Compute the link reliability matrix for a given configuration
    """
    if hamiltionian_list is not None: 
        assert len(partitions_set) == len(hamiltionian_list), "partitions_set and hamiltionian_list must have the same length"

    N = A.shape[0]
    link_reliability = np.zeros((N, N))
        
    if hamiltionian_list is not None:
        for i in range(N):
            for j in range(i, N):
                for k in range(len(partitions_set)):
                    link_reliability[i, j] += singleLinkReliability(A, partitions_set[k], i, j, hamiltionian_list[k])
    else:
        hamiltionian_list = []
        for i in range(N):
            for j in range(i, N):
                for k in range(len(partitions_set)):
                    l_r, H = singleLinkReliability(A, partitions_set[k], i, j, return_H=True)
                    link_reliability[i, j] += l_r
                    hamiltionian_list.append(H)


    # Symmetrize the matrix
    link_reliability = link_reliability + link_reliability.T - np.diag(np.diag(link_reliability))

    # Compute the partition function
    Z = np.sum(np.exp(-np.array(hamiltionian_list)))

    return link_reliability / Z


def sortLinkLists(links, not_links, link_reliability):
    """
    Sort the lists of links and not links according to their reliability
    """
    # Links: sort by increasing reliability
    links = sorted(links, key=lambda x: link_reliability[x[0], x[1]], reverse=False)
    # Not links: sort by decreasing reliability
    not_links = sorted(not_links, key=lambda x: link_reliability[x[0], x[1]], reverse=True)

    return links, not_links
   

def singleNetworkReliability(A, A_obs, groups, H=None, return_H=False):
    """
    Reliability of the network for a particular configuration
    """
    def exp_element(A, A_obs, groups, g1, g2):
        r = maxBetween(groups, g1, g2)
        l = linksBetween(A, groups, g1, g2)
        l_obs = linksBetween(A_obs, groups, g1, g2)

        t1 = (r+1)/(2*r+1)
        t2 = binom(r, l_obs)/binom(2*r, l+l_obs)

        return np.log(t1) + np.log(t2)

    if H is None: H = hamiltonian(A, groups)

    sum_h = 0
    for g2 in range(N):
        for g1 in range(g2+1):
            sum_h += exp_element(A, A_obs, groups, g1, g2)
          
    h = np.exp(sum_h)

    if return_H:
        return h*np.exp(-H), H
    else:
        return h*np.exp(-H)
    

def getNetworkReliability(A, A_obs, partitions_set, hamiltionian_list=None):
    """
    Compute the network reliability
    """
    if hamiltionian_list is not None: 
        assert len(partitions_set) == len(hamiltionian_list), "partitions_set and hamiltionian_list must have the same length"

    network_reliability = 0

    if hamiltionian_list is not None:
        for k in range(len(partitions_set)):
            network_reliability += singleNetworkReliability(A, A_obs, partitions_set[k], hamiltionian_list[k])
    else:
        hamiltionian_list = []
        for k in range(len(partitions_set)):
            n_r, H = singleNetworkReliability(A, A_obs, partitions_set[k], return_H=True)
            network_reliability += n_r
            hamiltionian_list.append(H)

    Z = np.sum(np.exp(-np.array(hamiltionian_list)))
    return network_reliability / Z

## Code

In [5]:
# Adjacency matrix (undirected graph with self-loops)
N = 30
upper_triangular = np.triu(np.random.randint(0, 2, size=(N, N)), 1)
A = upper_triangular + upper_triangular.T
np.fill_diagonal(A, np.random.randint(0, 2, size=N))

upper_triangular = np.triu(np.random.randint(0, 2, size=(N, N)), 1)
A_obs = upper_triangular + upper_triangular.T
np.fill_diagonal(A_obs, np.random.randint(0, 2, size=N))

In [6]:
# Get the Karate Club network
G = nx.karate_club_graph().to_undirected()
A = nx.adjacency_matrix(G).todense()
A = (A > 0).astype(int)
N = A.shape[0]

nx.draw(G, with_labels=True, node_color='skyblue', node_size=700, font_size=12, font_color='black', edge_color='gray')

### Trials

In [20]:
A_obs = corruptAdjacencyMatrix(A, p=0.4, mode='both')
A_obs

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 0],
       ...,
       [1, 1, 1, ..., 0, 1, 1],
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 1]])

In [25]:
# Groups data structure: list of lists
groups = [[] for _ in range(N)]

# Uniformly random initialization
for i in range(N): 
    g = np.random.randint(0, N)
    groups[g].append(i)

# Transient time (to test convergence)
H = hamiltonian(A_obs, groups)

transient_history = [H]

# Transient
transient = 10000
for k in range(transient):
    print("Progress: ", k, "/", transient, end="\r")
    groups, H = singleStep(groups, H, A_obs)

    transient_history.append(H)

In [26]:
plt.plot(transient_history)

In [25]:
# # Equilibrium (not in parallel)
# delay = 50 # for uncorrelate the samples
# n_samples = 20
# partitions_set = []

# for k in tqdm(range(n_samples)):
#     for _ in range(delay):
#         groups, H = singleStep(groups, H, A)
#     partitions_set.append(groups)

In [26]:
# Equilibrium: get the set of partitions
partitions_set, hamiltionian_list = parallelPartitionsSet(A, groups, n_samples=20, delay=20, n_cores=-1, return_H=True)

### Pipeline

In [7]:
# Currupt the adjacency matrix
A_obs = corruptAdjacencyMatrix(A, p=0.4, mode='both')
A_obs

array([[1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 1, 1],
       [1, 1, 0, ..., 1, 1, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 0, 1],
       [0, 1, 0, ..., 1, 1, 0]])

In [ ]:
# Initialize the current adjacency matrix
A_cur = A_obs.copy()

# Generate the set of partitions and the Hamiltonian list
partitions_set, hamiltionian_list = generatePartitionsSet(A_cur, n_samples=1000, delay=20, transient=500, n_cores=-1, return_H=True)

# Compute the network reliability and the link reliability matrix
R_N = getNetworkReliability(A_cur, A_obs, partitions_set, hamiltionian_list)
R_L = computeLinkReliabilityMatrix(A_cur, partitions_set, hamiltionian_list)
print("R_N: ", R_N)

# Sort the links (increasing R) and not links (decreasing R)
sorted_links, sorted_not_links = sortLinkLists(*getLinkLists(A_cur), R_L)

# Iterate to reconstruct the network
iterations = np.min([len(sorted_links), len(sorted_not_links)])
miss_update = 0
for k in range(np.min([len(sorted_links), len(sorted_not_links)])):
    # Choose a pair of link and not link in order
    link, not_link = sorted_links[k], sorted_not_links[k]

    # Swap the links
    A_temp = A_cur.copy()
    A_temp[link[0], link[1]] = 0
    A_temp[link[1], link[0]] = 0
    A_temp[not_link[0], not_link[1]] = 1
    A_temp[not_link[1], not_link[0]] = 1

    # Compute the network reliability with the new adjacency matrix
    partitions_set_temp, hamiltionian_list_temp = generatePartitionsSet(A_temp, n_samples=20, delay=20, transient=500, n_cores=-1, return_H=True)
    R_N_temp = getNetworkReliability(A_temp, A_obs, partitions_set_temp, hamiltionian_list_temp)

    # If the network reliability increases, update the adjacency matrix
    if R_N_temp > R_N:
        A_cur = A_temp
        R_N = R_N_temp
    else:
        miss_update += 1
    print("R_N: ", R_N)
        



In [30]:
### Trials and prints

def getNetworkReliability(A, A_obs, partitions_set, hamiltionian_list=None):
    """
    Compute the network reliability
    """
    if hamiltionian_list is not None: 
        assert len(partitions_set) == len(hamiltionian_list), "partitions_set and hamiltionian_list must have the same length"

    network_reliability = 0

    if hamiltionian_list is not None:
        for k in range(len(partitions_set)):
            network_reliability += singleNetworkReliability(A, A_obs, partitions_set[k], hamiltionian_list[k])
    else:
        hamiltionian_list = []
        for k in range(len(partitions_set)):
            n_r, H = singleNetworkReliability(A, A_obs, partitions_set[k], return_H=True)
            network_reliability += n_r
            hamiltionian_list.append(H)

    Z = np.sum(np.exp(-np.array(hamiltionian_list)))
    print(network_reliability, Z)
    return network_reliability / Z


def generatePartitionsSet(A, n_samples, delay, transient, n_cores=-1, return_H=True):
    """
    Generate the set of partitions from the adjacency matrix A
    """
    # Groups data structure: list of lists
    N = A.shape[0]
    groups = [[] for _ in range(N)]

    # Uniformly random initialization
    for i in range(N): 
        g = np.random.randint(0, N)
        groups[g].append(i)

    # Transient
    H = hamiltonian(A, groups)
    
    for _ in range(transient):
        groups, H = singleStep(groups, H, A)
    
    return parallelPartitionsSet(A, groups, n_samples, delay, n_cores, return_H)

In [8]:
%%time
# Initialize the current adjacency matrix
A_cur = A_obs.copy()

# Generate the set of partitions and the Hamiltonian list
partitions_set, hamiltionian_list = generatePartitionsSet(A_cur, n_samples=10e4, delay=20, transient=5000, n_cores=-1, return_H=True)


CPU times: user 12.3 s, sys: 192 ms, total: 12.5 s
Wall time: 8min 22s


In [38]:
hamiltionian_list

[np.float64(774.70794609913),
 np.float64(776.4174151141694),
 np.float64(773.1335376979941),
 np.float64(773.4733615633138),
 np.float64(765.8921918138406),
 np.float64(767.5540116605855),
 np.float64(766.7138041827877),
 np.float64(769.0114592008085),
 np.float64(764.9919990239763),
 np.float64(762.4533975276391),
 np.float64(762.6089030300026),
 np.float64(758.5930610228879),
 np.float64(757.915662199296),
 np.float64(760.1593718773084),
 np.float64(759.493529220229),
 np.float64(760.6708995593419),
 np.float64(761.4368986930231),
 np.float64(761.1076686620393),
 np.float64(765.024963215019),
 np.float64(758.1704686955197),
 np.float64(756.8588320232602),
 np.float64(754.053617840703),
 np.float64(753.9846249692162),
 np.float64(754.7890108827748),
 np.float64(754.4117166516334),
 np.float64(753.7387721783917),
 np.float64(755.6487312631605),
 np.float64(755.455996475883),
 np.float64(756.1103501428721),
 np.float64(760.4025434428114),
 np.float64(761.1594338787887),
 np.float64(754

In [39]:
getNetworkReliability(A_cur, A_obs, partitions_set, hamiltionian_list)

0.0 9.955e-321


np.float64(0.0)